In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

import requests
from bs4 import BeautifulSoup
# from sklearn.Logit_model_reloaded import score
# data_test = pd.read_csv("test_notes.csv", sep=',')

In [2]:
team_list_av = []
x_av = []
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://www.transfermarkt.fr/bundesliga/startseite/wettbewerb/L1"
pageTree = requests.get(page, headers=headers)
soup = BeautifulSoup(pageTree.content, 'html.parser')
body = soup.html.find('div',{'class':'large-8 columns'}).find_all('div',{'class':'box'})[1].find('div',{'class':'responsive-table'}).div.table.tbody.find_all('tr')

for g in body:

    team_list_av.append(g.find('td',{'hauptlink no-border-links show-for-small show-for-pad'}).a.text)
    x_av.append(g.find_all('td',{'class':'rechts show-for-small show-for-pad nowrap'})[1].text.replace(' mio. €','').replace(',','.'))

In [3]:
value_model_av = pd.DataFrame({'Team':team_list_av,'Value':x_av})
value_model_av.to_csv('value_model_av_bundes.csv',index=False)

In [4]:
value_model_af = pd.read_csv('value_model_av_bundes.csv',sep=',')

In [5]:
list_y = []
x = value_model_af['Value']
for i in x:
    list_y.append(0.012921312294796969*i + -2.906403333514497)

In [6]:
value_model = pd.DataFrame({'Team':value_model_af['Team'],'Value':value_model_af['Value'],'trend':list_y})

In [7]:
0.013651660777911766*2000+ -2.399976874866872

24.903344680956657

In [8]:
# Récupération du modèle de Régression Logistique 'lr'
with open('model_bun.pkl', 'rb') as f:
    my_unpickler = pickle.Unpickler(f)
    model_logit = my_unpickler.load()

    y_pred = model_logit.predict_proba(value_model[['trend']].values) 


In [9]:
for value in y_pred[0]:
    print(value)

0.00036777540771615325
0.9996322245922838


In [10]:
predictions = [round(value) for value in y_pred[0]]
# print(ytest)
# precision = model_logit.score(value_model[['Value','trend']],predictions)
# print(np.around(precision,decimals=2)*100,'%')

In [11]:
value_model

,Team,Value,trend
0,FC Bayern,852.90,8.114184
1,Bor. Dortmund,565.90,4.405767
2,RB Leipzig,505.55,3.625966
3,Bay. Leverkusen,352.75,1.651590
4,Bor. M'gladbach,300.00,0.969990
5,VfL Wolfsburg,287.83,0.812738
6,E. Frankfurt,231.25,0.081650
7,TSG Hoffenheim,207.65,-0.223293
8,VfB Stuttgart,169.63,-0.714561
9,Hertha BSC,137.05,-1.135537


In [13]:
# for i in prt_predict:
#     if i > 0.9:
#         print(np.around(i**2*(6/6),decimals=3))
#     else:
        
#         print(np.around(i*(6/6),decimals=3))

In [14]:
predict = []
prt_predict = []
for i in y_pred:
    prt_predict.append(i[1])
value_model['pourcentage'] = prt_predict
#print(value_model['pourcentage'])
proba = 6
count = 0
proba_list = []
high_count = 0
for a,k,i in zip(value_model.sort_values(by='pourcentage',ascending=False)['trend'],value_model.sort_values(by='pourcentage',ascending=False)['Team'],value_model.sort_values(by='pourcentage',ascending=False)['pourcentage']):
    if (proba == 6)&(a < 4):
        proba = proba - high_count
        
    proba_list.append(np.around(i*(proba/6),decimals=3))
    print(proba,count,high_count)
    count = count + 1
    if a > 4:
        high_count = high_count + 1
        continue

    elif proba < 3:
        if count == 10:
            proba = proba - 1
        else:
            continue
    else:
        proba = proba - 1

value_model['pourcentage'] = proba_list
        
for i in list(value_model['pourcentage']):
    if i < 0.5:
        predict.append(False)
    elif i > 0.5:
        predict.append(True)
    
#print(len(predict))
#print(len(y_pred[1]))
value_model['predict'] = predict

6 0 0
6 1 1
4 2 2
3 3 2
2 4 2
2 5 2
2 6 2
2 7 2
2 8 2
2 9 2
1 10 2
1 11 2
1 12 2
1 13 2
1 14 2
1 15 2
1 16 2
1 17 2


In [15]:
print('Prédictions :', value_model['Team'][0],' : ', np.around(y_pred[0][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][1],' : ', np.around(y_pred[1][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][2],' : ', np.around(y_pred[2][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][3],' : ', np.around(y_pred[3][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][4],' : ', np.around(y_pred[4][1],decimals=2),' %')

Prédictions : FC Bayern  :  1.0  %
Prédictions : Bor. Dortmund  :  0.98  %
Prédictions : RB Leipzig  :  0.97  %
Prédictions : Bay. Leverkusen  :  0.8  %
Prédictions : Bor. M'gladbach  :  0.67  %


In [17]:
value_model

,Team,Value,trend,pourcentage,predict
0,FC Bayern,852.90,8.114184,1.000,True
1,Bor. Dortmund,565.90,4.405767,0.985,True
2,RB Leipzig,505.55,3.625966,0.645,True
3,Bay. Leverkusen,352.75,1.651590,0.401,False
4,Bor. M'gladbach,300.00,0.969990,0.223,False
5,VfL Wolfsburg,287.83,0.812738,0.211,False
6,E. Frankfurt,231.25,0.081650,0.151,False
7,TSG Hoffenheim,207.65,-0.223293,0.126,False
8,VfB Stuttgart,169.63,-0.714561,0.090,False
9,Hertha BSC,137.05,-1.135537,0.065,False


In [18]:
value_model.to_csv('model_result_Bun.csv',index=False)